In [94]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from sklearn.utils import shuffle


In [164]:
train = pd.read_csv('/home/az/dev/atvisor/nlp/classifier/input/main_res.csv',encoding="ISO-8859-1")
test = pd.read_csv('/home/az/dev/atvisor/nlp/classifier/input/test.csv', encoding = "utf-8")

In [165]:
print('Total rows in test is {}'.format(len(test)))  
print('Total rows in train is {}'.format(len(train)))
print(train[cols_target].sum())

Total rows in test is 411
Total rows in train is 1026
category_access stairs                               3
category_alarms                                      5
category_ankle and wrist weights                     1
category_anti slip mats                              6
category_aprons and bibs                             5
category_arm slings                                  4
category_arm supports                                1
category_back support for bath                       1
category_bands and elastics                          1
category_bath lift pommel                            1
category_bath seats                                  5
category_bathing sponges                             1
category_bathing supports and stabilisers            1
category_bathrobes and towels                        3
category_bathroom caddy                              1
category_bathtub shelves                             1
category_bathtub slings                              1
category_ba

In [166]:
data = train[cols_target]   

In [167]:
def clean_text(text):
    text = text.lower()
    text = text.strip(' ')
    return text


In [170]:
train.head(5)
# train['title'] = train['title'].map(lambda com : clean_text(com))

,id,title,category_access stairs,category_alarms,category_ankle and wrist weights,category_anti slip mats,category_aprons and bibs,category_arm slings,category_arm supports,category_back support for bath,...,category_weighted blankets,category_wet wipes,category_whisks and brushes,category_white boards,category_whiteboard reminders,category_wipers,category_wrist hold-down and supports,category_wrist orthoses,category_zesters,category_zipper pullers
0,0,erase board,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1,plaid fold board,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2,planing whiteboards,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,white board,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,4,whiteboard reminders,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [171]:
X = train.title

In [172]:
test_X = test.title

In [173]:
print(X.shape, test_X.shape)


(1026,) (411,)


In [174]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=5000,stop_words='english')
vect

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [175]:
X_dtm = vect.fit_transform(X)

In [176]:
X_dtm

<1026x1007 sparse matrix of type '<class 'numpy.float64'>'
	with 2530 stored elements in Compressed Sparse Row format>

In [177]:
test_X_dtm = vect.transform(test_X)


In [178]:
test_X_dtm

<411x1007 sparse matrix of type '<class 'numpy.float64'>'
	with 1042 stored elements in Compressed Sparse Row format>

In [179]:
# import and instantiate the Logistic Regression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(C=12.0)

# create submission file
submission_binary = pd.read_csv('/home/az/dev/atvisor/nlp/classifier/input/test.csv')

for label in cols_target:
    print('... Processing {}'.format(label))
    y = train[label]
    # train the model using X_dtm & y
    logreg.fit(X_dtm, y)
    # compute the training accuracy
    y_pred_X = logreg.predict(X_dtm)
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    # compute the predicted probabilities for X_test_dtm
    test_y_prob = logreg.predict_proba(test_X_dtm)[:,1]
    submission_binary[label] = test_y_prob

... Processing category_access stairs
Training accuracy is 0.9990253411306043
... Processing category_alarms
Training accuracy is 0.9970760233918129
... Processing category_ankle and wrist weights
Training accuracy is 0.9990253411306043
... Processing category_anti slip mats
Training accuracy is 1.0
... Processing category_aprons and bibs
Training accuracy is 1.0
... Processing category_arm slings
Training accuracy is 0.9990253411306043
... Processing category_arm supports
Training accuracy is 0.9990253411306043
... Processing category_back support for bath
Training accuracy is 0.9990253411306043
... Processing category_bands and elastics
Training accuracy is 0.9990253411306043
... Processing category_bath lift pommel
Training accuracy is 0.9990253411306043
... Processing category_bath seats
Training accuracy is 1.0
... Processing category_bathing sponges
Training accuracy is 0.9990253411306043
... Processing category_bathing supports and stabilisers
Training accuracy is 0.999025341130

Training accuracy is 1.0
... Processing category_cognitive support tools
Training accuracy is 0.9990253411306043
... Processing category_color identifiers
Training accuracy is 0.9990253411306043
... Processing category_combs and hairbrushes
Training accuracy is 0.9990253411306043
... Processing category_commode chairs
Training accuracy is 1.0
... Processing category_communication devices
Training accuracy is 0.9970760233918129
... Processing category_computer mice
Training accuracy is 0.9980506822612085
... Processing category_continence controller
Training accuracy is 0.9990253411306043
... Processing category_cookware
Training accuracy is 0.9990253411306043
... Processing category_corkscrews
Training accuracy is 0.9990253411306043
... Processing category_crutches
Training accuracy is 1.0
... Processing category_cup holders
Training accuracy is 0.9980506822612085
... Processing category_cup warmers
Training accuracy is 0.9990253411306043
... Processing category_cupboard organization u

Training accuracy is 0.9990253411306043
... Processing category_faucets
Training accuracy is 1.0
... Processing category_feeder seats
Training accuracy is 0.9990253411306043
... Processing category_feeding aids
Training accuracy is 1.0
... Processing category_fidget toys
Training accuracy is 0.9990253411306043
... Processing category_finders
Training accuracy is 0.9990253411306043
... Processing category_fixation products
Training accuracy is 0.9980506822612085
... Processing category_flood guards
Training accuracy is 0.9990253411306043
... Processing category_floor mopping equipment
Training accuracy is 1.0
... Processing category_floor seats
Training accuracy is 0.9980506822612085
... Processing category_food blenders and mixers
Training accuracy is 0.9990253411306043
... Processing category_food guards
Training accuracy is 0.9990253411306043
... Processing category_food holders
Training accuracy is 0.9980506822612085
... Processing category_food openers
Training accuracy is 1.0
... 

Training accuracy is 0.9990253411306043
... Processing category_hoses
Training accuracy is 0.9990253411306043
... Processing category_hot bottles
Training accuracy is 0.9990253411306043
... Processing category_ice trays
Training accuracy is 0.9990253411306043
... Processing category_incontinence alarms
Training accuracy is 0.9990253411306043
... Processing category_incontinence bed pads
Training accuracy is 0.9990253411306043
... Processing category_incontinence collectors
Training accuracy is 0.9990253411306043
... Processing category_incontinence garments
Training accuracy is 0.9980506822612085
... Processing category_incontinence stimulators
Training accuracy is 0.9990253411306043
... Processing category_induction hobs
Training accuracy is 0.9990253411306043
... Processing category_inflatable adult bathtub
Training accuracy is 0.9990253411306043
... Processing category_inserts and shortners
Training accuracy is 1.0
... Processing category_ironing instruments
Training accuracy is 1.0

Training accuracy is 0.9990253411306043
... Processing category_mashers
Training accuracy is 0.9990253411306043
... Processing category_massagers
Training accuracy is 0.9990253411306043
... Processing category_mattresses
Training accuracy is 0.9990253411306043
... Processing category_measuring cups
Training accuracy is 0.9990253411306043
... Processing category_measuring jugs
Training accuracy is 0.9990253411306043
... Processing category_measuring scoops
Training accuracy is 0.9990253411306043
... Processing category_measuring spoons
Training accuracy is 0.9990253411306043
... Processing category_medication guides
Training accuracy is 1.0
... Processing category_medication reminders
Training accuracy is 1.0
... Processing category_medication spoons
Training accuracy is 0.9990253411306043
... Processing category_medication temperature keepers
Training accuracy is 0.9990253411306043
... Processing category_mirrors
Training accuracy is 1.0
... Processing category_mobile phones
Training a

Training accuracy is 0.9980506822612085
... Processing category_pads
Training accuracy is 0.9990253411306043
... Processing category_pans
Training accuracy is 0.9980506822612085
... Processing category_peelers
Training accuracy is 0.9990253411306043
... Processing category_pen
Training accuracy is 0.9990253411306043
... Processing category_pencil holders
Training accuracy is 0.9990253411306043
... Processing category_personal safety products
Training accuracy is 1.0
... Processing category_personal scales
Training accuracy is 1.0
... Processing category_personal sound amplifier
Training accuracy is 0.9990253411306043
... Processing category_pet care reminders
Training accuracy is 0.9990253411306043
... Processing category_pet feeders
Training accuracy is 0.9990253411306043
... Processing category_pickers
Training accuracy is 0.9990253411306043
... Processing category_pill dispensers
Training accuracy is 1.0
... Processing category_pillows
Training accuracy is 0.9980506822612085
... Pro


... Processing category_salt and pepper shakers
Training accuracy is 0.9990253411306043
... Processing category_scan readers
Training accuracy is 0.9990253411306043
... Processing category_scan writers
Training accuracy is 0.9990253411306043
... Processing category_scarves
Training accuracy is 0.9990253411306043
... Processing category_scissors
Training accuracy is 0.9990253411306043
... Processing category_sealers
Training accuracy is 0.9990253411306043
... Processing category_seat covers
Training accuracy is 0.9990253411306043
... Processing category_seating cushions
Training accuracy is 0.9990253411306043
... Processing category_sensory integration products
Training accuracy is 0.9970760233918129
... Processing category_shampooing aids
Training accuracy is 1.0
... Processing category_shears
Training accuracy is 0.9990253411306043
... Processing category_sheets
Training accuracy is 0.9990253411306043
... Processing category_shirts
Training accuracy is 1.0
... Processing category_sho

Training accuracy is 0.9990253411306043
... Processing category_steps
Training accuracy is 0.9990253411306043
... Processing category_stimulating lights
Training accuracy is 0.9980506822612085
... Processing category_stocking aids
Training accuracy is 0.9990253411306043
... Processing category_stools
Training accuracy is 1.0
... Processing category_strainers
Training accuracy is 0.9990253411306043
... Processing category_straws
Training accuracy is 1.0
... Processing category_suits
Training accuracy is 1.0
... Processing category_swimwear
Training accuracy is 0.9980506822612085
... Processing category_tv listening speaker
Training accuracy is 0.9990253411306043
... Processing category_tags
Training accuracy is 0.9990253411306043
... Processing category_tap turners
Training accuracy is 0.9990253411306043
... Processing category_tapes
Training accuracy is 1.0
... Processing category_taps
Training accuracy is 1.0
... Processing category_task management and assistance tools
Training accura

Training accuracy is 0.9990253411306043
... Processing category_towels bars and stands
Training accuracy is 0.9990253411306043
... Processing category_transfer aids
Training accuracy is 1.0
... Processing category_transfer lift
Training accuracy is 1.0
... Processing category_trays
Training accuracy is 1.0
... Processing category_trolleys
Training accuracy is 1.0
... Processing category_trouser aids
Training accuracy is 0.9990253411306043
... Processing category_tube squeezers
Training accuracy is 0.9990253411306043
... Processing category_turning aids
Training accuracy is 0.9990253411306043
... Processing category_tweezers
Training accuracy is 1.0
... Processing category_umbrellas
Training accuracy is 0.9990253411306043
... Processing category_underwear
Training accuracy is 1.0
... Processing category_urinals
Training accuracy is 1.0
... Processing category_utensils
Training accuracy is 0.9990253411306043
... Processing category_vegetable brushes
Training accuracy is 0.999025341130604

In [180]:
submission_binary.head()

,id,title,category_access stairs,category_alarms,category_ankle and wrist weights,category_anti slip mats,category_aprons and bibs,category_arm slings,category_arm supports,category_back support for bath,...,category_weighted blankets,category_wet wipes,category_whisks and brushes,category_white boards,category_whiteboard reminders,category_wipers,category_wrist hold-down and supports,category_wrist orthoses,category_zesters,category_zipper pullers
0,615,men's hunting pants,0.001497,0.001910,0.001016,0.001324,0.002320,0.001570,0.000984,0.000932,...,0.001099,0.001030,0.001473,0.002176,0.000964,0.001030,0.001198,0.001233,0.001030,0.001602
1,616,men's snowpants,0.001508,0.001915,0.001021,0.001332,0.002351,0.001583,0.000989,0.000936,...,0.001105,0.001035,0.001484,0.002203,0.000969,0.001035,0.001205,0.001240,0.001035,0.001616
2,617,men's sweatpants,0.001481,0.001871,0.001008,0.001311,0.002285,0.001552,0.000977,0.000925,...,0.001090,0.001022,0.001458,0.002145,0.000958,0.001022,0.001187,0.001222,0.001022,0.001584
3,618,men's wheelchair trousers,0.001429,0.001785,0.000985,0.001271,0.002155,0.001495,0.000955,0.000906,...,0.001062,0.000998,0.001407,0.002031,0.000936,0.000998,0.001154,0.001186,0.000998,0.001524
4,619,rain pants,0.001632,0.002130,0.001078,0.001429,0.002641,0.001718,0.001043,0.000985,...,0.001172,0.001094,0.001603,0.002458,0.001021,0.001094,0.001285,0.001324,0.001094,0.001757


In [181]:
submission_binary.to_csv('submission_binary.csv',index=False)

In [182]:
submission_binary.describe()

,id,category_access stairs,category_alarms,category_ankle and wrist weights,category_anti slip mats,category_aprons and bibs,category_arm slings,category_arm supports,category_back support for bath,category_bands and elastics,...,category_weighted blankets,category_wet wipes,category_whisks and brushes,category_white boards,category_whiteboard reminders,category_wipers,category_wrist hold-down and supports,category_wrist orthoses,category_zesters,category_zipper pullers
count,411.00000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,...,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000,411.000000
mean,820.00000,0.001636,0.002335,0.001060,0.001503,0.009417,0.001826,0.001158,0.001136,0.001075,...,0.001171,0.001075,0.001740,0.002423,0.001003,0.001075,0.001291,0.001296,0.001075,0.005799
std,118.78973,0.000501,0.001861,0.000050,0.000992,0.061918,0.001193,0.002281,0.001658,0.000052,...,0.000338,0.000052,0.000981,0.001263,0.000046,0.000052,0.000364,0.000075,0.000052,0.047891
min,615.00000,0.001270,0.001540,0.000906,0.000503,0.001814,0.001322,0.000785,0.000711,0.000918,...,0.000917,0.000918,0.001216,0.001387,0.000865,0.000918,0.001048,0.001075,0.000918,0.001345
25%,717.50000,0.001509,0.001904,0.001023,0.001334,0.002354,0.001583,0.000987,0.000934,0.001038,...,0.001105,0.001038,0.001491,0.002182,0.000971,0.001038,0.001205,0.001240,0.001038,0.001619
50%,820.00000,0.001610,0.002075,0.001068,0.001411,0.002578,0.001695,0.001030,0.000975,0.001084,...,0.001159,0.001084,0.001591,0.002402,0.001011,0.001084,0.001271,0.001309,0.001084,0.001725
75%,922.50000,0.001685,0.002208,0.001102,0.001474,0.002773,0.001783,0.001065,0.001004,0.001118,...,0.001200,0.001118,0.001664,0.002575,0.001042,0.001118,0.001317,0.001359,0.001118,0.001817
max,1025.00000,0.007551,0.020374,0.001127,0.018410,0.645533,0.021649,0.046586,0.024926,0.001144,...,0.006527,0.001144,0.010300,0.027425,0.001064,0.001144,0.006701,0.001397,0.001144,0.605662
